In [0]:
spark.conf.set(
  "fs.azure.account.key.assignmentstoreacc.dfs.core.windows.net",
  "<SA-Key>"
)


In [0]:
product_df = spark.read.format("delta").load(
    "abfss://silver@assignmentstoreacc.dfs.core.windows.net/sales_view/product"
)

store_df = spark.read.format("delta").load(
    "abfss://silver@assignmentstoreacc.dfs.core.windows.net/sales_view/store"
)

sales_df = spark.read.format("delta").load(
    "abfss://silver@assignmentstoreacc.dfs.core.windows.net/sales_view/customer_sales"
)


In [0]:
product_df.select("store_id").printSchema()
store_df.select("store_id").printSchema()


In [0]:
product_df.select("store_id").distinct().show(20, False)
store_df.select("store_id").distinct().show(20, False)


In [0]:
sales_df = sales_df.withColumnRenamed("product__id", "product_id")


In [0]:
final_df = sales_df \
    .join(product_df, on="product_id", how="left") \
    .join(store_df, on="store_id", how="left")


display(final_df)

In [0]:
from pyspark.sql.functions import col

gold_df = final_df.select(col("order_date").alias("OrderDate"),col("category_id").alias("Category"),col("city").alias("City"),
    col("customer_id").alias("CustomerID"),col("order_id").alias("OrderID"),
    col("product_id").alias("ProductID"),col("profit").alias("Profit"),col("region").alias("Region"),
    col("sales").alias("Sales"),col("segment").alias("Segment"),col("ship_date").alias("ShipDate"),col("ship_mode").alias("ShipMode"),
    col("latitude"),col("longitude"),col("store_name"),col("location"),
    col("manager_name"),col("product_name"),col("price"),
    col("stock_quantity"),col("image_url")
)

display(gold_df)

In [0]:
gold_path = "abfss://gold@assignmentstoreacc.dfs.core.windows.net/sales_view/StoreProductSalesAnalysis"

gold_df.write.format("delta").mode("overwrite").save(gold_path)


In [0]:
spark.read.format("delta").load(gold_path).display()
